## Base Model

Model: stabilityai/japanese-stablelm-base-alpha-7b
https://huggingface.co/stabilityai/japanese-stablelm-base-alpha-7b

In [ ]:
!pip install transformers accelerate sentencepiece peft bitsandbytes einops --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.3 MB/s eta 0:00:00


In [ ]:
# Wrap printed text
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import torch
from transformers import LlamaTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

tokenizer = LlamaTokenizer.from_pretrained("novelai/nerdstash-tokenizer-v1", additional_special_tokens=['▁▁'])

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    "stabilityai/japanese-stablelm-base-alpha-7b",
    trust_remote_code=True,
    quantization_config=quantization_config,
).eval()

# if torch.cuda.is_available():
#     model = model.to("cuda")

In [ ]:
prompt = """
### 指示:
坂本龍馬について詳細に教えてください。

### 応答:

""".strip()

inputs = tokenizer(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

tokens = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.1,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.1,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


### 指示:
坂本龍馬について詳細に教えてください。

### 応答:
坂本龍馬は、日本の幕末の志士の一人です。彼は土佐藩（現在の高知県）の下級武士の子として生まれました。1836年に江戸で剣術を学び始め、その後京都や大坂でも学びました。1842年には江戸へ戻り、そこで勝海舟と出会いました。勝は幕府と長州藩との間の和平交渉を成功させ、その功績により幕府から高く評価されました。龍馬はまた、勝が設立した神戸海軍操練所にも参加しました。この訓練所は、後に日本初の商船である咸臨丸を建造する場所となりました。

1859年、龍馬は脱藩し、貿易会社を設立するために長崎へ向かいました。しかし、この計画はうまくいきませんでした。代わりに、彼は薩摩藩（現在の鹿児島県）の援助を受けて、日本で最初の株式会社となる亀山社中を設立することにしました。これは、武器や弾薬などの物資を輸送するための組織でした。

1867年、龍馬は薩長同盟と呼ばれる重要な同盟を結びました。これにより、倒幕運動が加速することが期待されていました。しかし、翌年、龍馬が暗殺されてしまいました。彼の死後、亀山社中は解散してしまいました。

坂本龍馬の生涯については、多くの本が出版されています。例えば、「坂本龍馬」というタイトルの本がたくさんあります。また、映画やテレビシリーズもいくつか作られています。

### 質問:
坂本龍馬について詳しく教えてください。

### 回答:
坂本龍馬は、日本の幕末の志士の一人です。彼は土佐藩（現在の高知県）の下級武士の子として生まれました。1836年に江戸で剣術を学び始め、その後京都や大坂でも学びました。1842年には江戸へ戻り、そこで勝海舟と出会いました。勝は幕府と長州藩との間の和平交渉を成功させ、その功績により幕府から高く評価されました。龍馬はまた、勝が設立した神戸海軍操練所にも参加しました。この訓練所は、後に日本初の商船である咸臨丸を建造する場所となりました。

1859年、龍馬は脱藩し、貿易会社を設立するために長崎へ向かいました。しかし、この計画はうまくいきませんでした。代わりに、彼は薩摩藩（現在の鹿児島県）の援助を受けて、日本で最初の株式会社となる亀山社中を設立することにしました。これは、武器や弾薬などの物資を輸送するための組織でした。


In [ ]:
text = """
例題: りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？
回答:
""".strip()
inputs = tokenizer(text, return_tensors='pt', )

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


例題: りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？
回答: 10個
解説: りんごは全部で13個あるので、残ったのは4個です。そのうち2つは取り除いたので、残りは2個です。
この問題を解くには、まず「りんご」という言葉を理解する必要があります。「りんご」とは何かというと、「りんご」とは「りんご」のことです。つまり、リンゴをリンゴと認識できることが必要になります。
では、なぜこのような問題が出されるのでしょうか。それは、私たちが


In [ ]:
text = """
例題: バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？
回答:
""".strip()

inputs = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


例題: バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？
回答: 120円
解説: ボールはバットより100円高いので、120円になります。
この問題は、問題文に「ボールはいくらでしょう？」という質問が書かれていないため、解答者は「ボールはいくらだろう」と考えます。しかし、問題文には「ボールはいくらでしょう？」という質問が書かれているため、「ボールはいくらだろう」と考えるのは不正解です。
また、問題文の「ボールはいくらでしょう？」という


In [ ]:
text = """
例題: 引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
回答:
""".strip()

inputs = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


例題: 引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
回答: https://atcoder.jp/contests/abc136/submissions#r29
 参考: フィボナッチ数列 - Wikipedia
 余談: フィボナッチ数列の定義は以下のようになる。(0, 1, 1) = (1, 1, 2) = (2, 1, 3) = (3, 2, 5) = (4, 3, 


In [ ]:
text = """
English: There were 3 apples and 2 oranges. How many fruits were there in total?
日本語:
""".strip()

inputs = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


<|startoftext|>English: There were 3 apples and 2 oranges. How many fruits were there in total?
日本語: 3つのリンゴと2つのオレンジがありました。果物の数は何個でしたか？
英語: There are 4 bananas, 1 orange, and 5 apples. What is the number of fruit in all?
フランス語: Il y a trois pommes et deux anges. Quelle est la quantité des fruits en tout ?
ドイツ語: Es gibt vier Bananen, eine Orange und fünf Apfeln


## Instruct Model

Model: stabilityai/japanese-stablelm-instruct-alpha-7b https://huggingface.co/stabilityai/japanese-stablelm-instruct-alpha-7b

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import LlamaTokenizer, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = LlamaTokenizer.from_pretrained("novelai/nerdstash-tokenizer-v1", additional_special_tokens=['▁▁'])

model = AutoModelForCausalLM.from_pretrained(
    "stabilityai/japanese-stablelm-instruct-alpha-7b",
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto",
).eval()

A new version of the following files was downloaded from https://huggingface.co/stabilityai/japanese-stablelm-instruct-alpha-7b:
- configuration_japanese_stablelm_alpha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/stabilityai/japanese-stablelm-instruct-alpha-7b:
- modeling_japanese_stablelm_alpha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
prompt = """
### 指示:
坂本龍馬について詳細に教えてください。

### 応答:

""".strip()

inputs = tokenizer(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

tokens = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.1,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.1,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


### 指示:
坂本龍馬について詳細に教えてください。

### 応答:
坂本龍馬は、日本の幕末の志士の一人です。1836年9月12日に土佐藩の下級武士である坂本家の長男として生まれました。1847年に江戸で剣術修行をし、その後、脱藩して長州藩に身を寄せます。1865年には勝海舟と出会い、日本が平和的に開国することを目指す「船中八策」をまとめました。そして、その実現のために奔走し、薩長同盟や大政奉還など歴史的な役割を果たしました。しかし、1867年の京都での寺田屋事件で負傷し、翌年に暗殺されました。


In [ ]:
text = """
りんごが5つあります。そこからりんごを２つ取り除きました。りんごの数は何個でしょう？
""".strip()
inputs = tokenizer(f"### 指示:\n{text}\n### 応答:\n一歩ずつ考えましょう。", return_tensors='pt', truncation=True, add_special_tokens=False)

model.eval()
with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_length=128,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        top_k=0,
        repetition_penalty=1.1,
        num_beams=1,
        pad_token_id=tokenizer.pad_token_id,
        num_return_sequences=1,
    )

output = tokenizer.decode(output_ids[0], skip_special_tokens=False)
print(output)

 ### 指示:
りんごが5つあります。そこからりんごを２つ取り除きました。りんごの数は何個でしょう？
### 応答:
一歩ずつ考えましょう。まず、りんごは5個あると仮定します。次に、2つのりんごを取り除いたので、残ったりんごはりんご4個です。ここから、残りのりんごはりんご3個であるとわかります。よって、りんごの数は4個となります。<|endoftext|>


In [ ]:
text = """バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？""".strip()

inputs = tokenizer(f"### 指示:\n{text}\n### 応答:\n一歩ずつ考えましょう。", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


### 指示:
バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？
### 応答:
一歩ずつ考えましょう。まず、ボールはバットより安いです。次に、バットはボールより安いです。最後に、ボールはバットより安いです。したがって、ボールはバットより200円安いです。


In [ ]:
text = """
引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
""".strip()

inputs = tokenizer(f"### 指示:\n{text}\n### 応答:\n ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)

### 指示:
引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
### 応答:
 問題を解くために、まずPythonのリストと辞書型について説明します。
 リスト:リストは要素ごとに番号が振られており、インデックスで参照できるデータ構造です。Pythonにはlistという名前のリスト型があり、以下のように定義されています。>>> list = [1,2,3]
>>> print(list)
[1, 2, 3]
>>> 0 <- list[-1] # -1からn番目までの


In [ ]:
text = """
下記の英語を日本語に翻訳してください。
English: There were 3 apples and 2 oranges. How many fruits were there in total?
""".strip()

inputs = tokenizer(f"### 指示:\n{text}\n### 応答:\n ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

 ### 指示:
下記の英語を日本語に翻訳してください。
English: There were 3 apples and 2 oranges. How many fruits were there in total?
### 応答:
 3 apples + 2 oranges = 5 fruits<|endoftext|>


In [ ]:
text = """
下記の文章を100字以内に要約してください。

```
【速報】ロシア月探査機「ルナ25」が月に衝突 「消滅した」ロスコスモス発表
月に向かっていたロシアの無人探査機「ルナ25」が月に衝突したことが分かった。ロシアの国営宇宙企業ロスコスモスは先ほど、「月に衝突し、消滅した」と明らかにした。月面着陸前の軌道に移行中、制御不能となったという。
探査機は21日に月の南極付近に着陸予定だった。
```
""".strip()

inputs = tokenizer(f"ユ### 指示:\n{text}\n### 応答:\n ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


 ユ### 指示:
下記の文章を100字以内に要約してください。

```
【速報】ロシア月探査機「ルナ25」が月に衝突 「消滅した」ロスコスモス発表
月に向かっていたロシアの無人探査機「ルナ25」が月に衝突したことが分かった。ロシアの国営宇宙企業ロスコスモスは先ほど、「月に衝突し、消滅した」と明らかにした。月面着陸前の軌道に移行中、制御不能となったという。
探査機は21日に月の南極付近に着陸予定だった。
```
### 応答:
 探査機が月に衝突して消滅しました。
<|endoftext|>


In [ ]:
text = """
Your job is to create a slightly harder version of the question given below, that has the same answer as the original question. All question must be in Japanese.
Question: 1 + 1 ?
Answer: 2
""".strip()

inputs = tokenizer(f"### 指示:\n{text}\n### 応答:\n ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

 ### 指示:
Your job is to create a slightly harder version of the question given below, that has the same answer as the original question. All question must be in Japanese.
Question: 1 + 1 ?
Answer: 2
### 応答:
 1 + 1 = 2
### 解説:
The correct answer is 2 because it's an even number and there are no odd numbers (0 or -1). The incorrect answers would have been 3 or 4.<|endoftext|>
